# Check if representative group knockoffs are working

In [5]:
# load packages for this tutorial
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using Distributions
using Clustering
using ProgressMeter
using LowRankApprox
using Plots
using CSV, DataFrames
gr(fmt=:png);

# some helper functions to compute power and empirical FDR
function TP(correct_groups, signif_groups)
    return length(signif_groups ∩ correct_groups) / max(1, length(correct_groups))
end
function TP(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return TP(correct_groups, signif_groups)
end
function power(correct_snps, discovered_snps)
    return length(discovered_snps ∩ correct_snps) / length(correct_snps)
end
function FDR(correct_groups, signif_groups)
    FP = length(signif_groups) - length(signif_groups ∩ correct_groups) # number of false positives
    return FP / max(1, length(signif_groups))
end
function FDR(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return FDR(correct_groups, signif_groups)
end
function get_signif_groups(β, groups)
    correct_groups = Int[]
    for i in findall(!iszero, β)
        g = groups[i]
        g ∈ correct_groups || push!(correct_groups, g)
    end
    return correct_groups
end

get_signif_groups (generic function with 1 method)

## gnomdAD panel

In [169]:
datadir = "/Users/biona001/Benjamin_Folder/research/4th_project/group_knockoff_test_data"
covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
Σ = covfile |> Matrix{Float64}
Σ = 0.99Σ + 0.01I #ensure PSD

# test on smaller data
idx = 120 # 1261 # includes largest group with 192 members
Σ = Σ[1:idx, 1:idx];

In [346]:
# simulate data
m = 1
p = size(Σ, 1)
k = 10 # number of causal groups
n = 2250 # sample size

# simulate X
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups empirically
nrep = 5
groups, group_reps = hc_partition_groups(X, rep_method=:rss, nrep=nrep, force_contiguous=true)
unique_groups = unique(groups)
countmap(groups) |> values |> collect |> sort

# rep group knockoffs
# @time ko = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)
@time Xr, Xc, X̃r, X̃c, X̃r_correct, X̃c_correct = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)

size(S) = (45, 45)
size(X̃new) = (2250, 120)
size(X̃r) = (2250, 45)
size(X̃c) = (2250, 75)
  0.526004 seconds (1.29 M allocations: 82.961 MiB, 93.48% compilation time)


([-0.5110675451099052 -1.3881943400814263 … -1.9264745182523682 -0.11326532734756863; 0.17372098398962296 -0.7204067030935772 … -0.6009914632934862 0.35745984835709765; … ; 0.12118598787834965 0.022179897316268994 … -1.7890884233928623 0.9393280181137896; 1.4285314677916803 0.9127413126386185 … -0.45577801281767943 1.3688961831461546], [0.6140871237716775 0.2358862820866552 … -0.7026932784158375 -0.6486003604037484; -0.46497302210206676 -1.036864004455972 … 0.5758547150426133 0.4507170039909755; … ; -1.6365292856257758 -1.3517240266918182 … 1.0810421690581342 1.0131041879374327; -0.8019185702810434 -0.35232749085755083 … 1.127226215527613 0.5714755877476627], [1.6920487937665598 0.9418001642126749 … -0.36841374135996735 -0.6975913793229805; 2.3062224906991537 1.2924126153325421 … -0.4734702905316992 -0.7438856246181085; … ; 1.14823670796934 -1.6812958304573111 … 1.0241997379559966 0.8188759587097809; -0.12827498032476303 0.5361584193843015 … -0.17294769760102513 -0.31896002249160504], 

In [347]:
cor(Xr)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         0.0214562     0.343461   …   0.151049    0.151193    0.173956
  0.0214562   1.0           0.0654205      0.510809    0.512775    0.352004
  0.343461    0.0654205     1.0            0.251284    0.247053    0.354849
 -0.143897   -0.000279531  -0.098534      -0.032797   -0.0324325  -0.0383607
  0.0534765   0.0132109     0.108973      -0.0415864  -0.0427993  -0.0230774
 -0.0809927  -0.000432971  -0.0758806  …  -0.0458723  -0.0470956  -0.0588285
  0.143861    0.515923      0.245675       0.982402    0.981172    0.743078
  0.151049    0.510809      0.251284       1.0         0.988296    0.745463
  0.151193    0.512775      0.247053       0.988296    1.0         0.741921
  0.173956    0.352004      0.354849       0.745463    0.741921    1.0

In [348]:
cor(X̃r_correct)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         -0.00082746   0.375064   …   0.106726    0.106836    0.151087
 -0.00082746   1.0          0.0754255      0.494888    0.49177     0.320406
  0.375064     0.0754255    1.0            0.241302    0.241689    0.337306
 -0.117003    -0.0390905   -0.0750819     -0.0446874  -0.0454385  -0.040415
  0.0897734    0.00776567   0.121324      -0.0469428  -0.04681    -0.0386682
 -0.0916582   -0.0271406   -0.0768169  …  -0.0715908  -0.0743989  -0.0833821
  0.106772     0.495657     0.244662       0.981857    0.98059     0.737007
  0.106726     0.494888     0.241302       1.0         0.987824    0.737103
  0.106836     0.49177      0.241689       0.987824    1.0         0.738291
  0.151087     0.320406     0.337306       0.737103    0.738291    1.0

In [349]:
cor(X̃r)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         0.0289542     0.352674   …   0.159105    0.159503    0.199701
  0.0289542   1.0           0.0539519      0.49219     0.488663    0.290543
  0.352674    0.0539519     1.0            0.22175     0.22291     0.314301
 -0.0980353  -0.0431034    -0.0375986     -0.0466248  -0.0427135  -0.0469491
  0.0775284   0.000834834   0.131398      -0.0114601  -0.0110098  -0.0315257
 -0.115678   -0.0251386    -0.0272556  …  -0.0474358  -0.046448   -0.0333951
  0.14983     0.495826      0.222871       0.982097    0.980983    0.730456
  0.159105    0.49219       0.22175        1.0         0.987848    0.730251
  0.159503    0.488663      0.22291        0.987848    1.0         0.728424
  0.199701    0.290543      0.314301       0.730251    0.728424    1.0

In [350]:
cor(Xc)[1:10, 1:10]

10×10 Matrix{Float64}:
 1.0       0.788392  0.788917  0.789835  …  0.781022  0.61717   0.595768
 0.788392  1.0       0.971427  0.966379     0.959829  0.768037  0.748935
 0.788917  0.971427  1.0       0.977029     0.971187  0.77657   0.75579
 0.789835  0.966379  0.977029  1.0          0.965998  0.772552  0.752934
 0.793918  0.972304  0.975535  0.970434     0.965986  0.768792  0.748792
 0.782819  0.957139  0.966561  0.962649  …  0.958409  0.764097  0.744614
 0.789333  0.971406  0.9817    0.973904     0.973171  0.775915  0.754788
 0.781022  0.959829  0.971187  0.965998     1.0       0.769265  0.748948
 0.61717   0.768037  0.77657   0.772552     0.769265  1.0       0.947376
 0.595768  0.748935  0.75579   0.752934     0.748948  0.947376  1.0

In [351]:
cor(X̃c_correct)[1:10, 1:10]

10×10 Matrix{Float64}:
 1.0       0.798601  0.79345   0.792286  …  0.785537  0.626591  0.60999
 0.798601  1.0       0.972888  0.964586     0.961216  0.770493  0.752837
 0.79345   0.972888  1.0       0.976472     0.971012  0.776289  0.757283
 0.792286  0.964586  0.976472  1.0          0.966009  0.774515  0.754277
 0.801894  0.973648  0.975496  0.971373     0.967522  0.772426  0.751844
 0.787518  0.956767  0.966791  0.963852  …  0.959769  0.759888  0.741857
 0.800609  0.970971  0.981349  0.97552      0.973309  0.778961  0.76004
 0.785537  0.961216  0.971012  0.966009     1.0       0.776631  0.755402
 0.626591  0.770493  0.776289  0.774515     0.776631  1.0       0.945062
 0.60999   0.752837  0.757283  0.754277     0.755402  0.945062  1.0

In [352]:
cor(X̃c)[1:10, 1:10]

10×10 Matrix{Float64}:
 1.0       0.973618  0.978192  0.975807  …  0.850477  0.851013  0.846093
 0.973618  1.0       0.980652  0.976538     0.856494  0.856242  0.85052
 0.978192  0.980652  1.0       0.982891     0.862299  0.861864  0.856392
 0.975807  0.976538  0.982891  1.0          0.860908  0.86134   0.856676
 0.953422  0.955627  0.961947  0.962862     0.841597  0.84169   0.834889
 0.846504  0.853645  0.85806   0.858054  …  0.975124  0.974632  0.976257
 0.839215  0.845762  0.850517  0.849795     0.960791  0.95935   0.96689
 0.850477  0.856494  0.862299  0.860908     1.0       0.982611  0.974843
 0.851013  0.856242  0.861864  0.86134      0.982611  1.0       0.973316
 0.846093  0.85052   0.856392  0.856676     0.974843  0.973316  1.0

In [194]:
# simulate data
m = 1
p = size(Σ, 1)
k = 10 # number of causal groups
n = 2250 # sample size

# simulate X
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups empirically
nrep = 1
groups, group_reps = id_partition_groups(X, rep_method=:rss, nrep=nrep)
unique_groups = unique(groups)
countmap(groups) |> values |> collect |> sort

# rep group knockoffs
@time ko = modelX_gaussian_group_knockoffs(X, :maxent, groups, μ, Σ, m=m);

  0.189736 seconds (14.79 k allocations: 17.020 MiB)


In [182]:
cor(X)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         0.0136149   0.343179   …   0.117435    0.11872     0.124917
  0.0136149   1.0         0.0805718      0.503197    0.504936    0.322599
  0.343179    0.0805718   1.0            0.251345    0.25585     0.337278
 -0.132733   -0.0676995  -0.0636298     -0.0694679  -0.0693109  -0.0431528
  0.0807121  -0.0274369   0.110185      -0.072626   -0.0690193  -0.0710977
 -0.112806   -0.0316348  -0.0581683  …  -0.050345   -0.0469703  -0.0304699
  0.119676    0.50619     0.255603       0.981795    0.981342    0.716044
  0.117435    0.503197    0.251345       1.0         0.988406    0.714763
  0.11872     0.504936    0.25585        0.988406    1.0         0.71204
  0.124917    0.322599    0.337278       0.714763    0.71204     1.0

In [183]:
cor(ko.X̃)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0          -0.000374575  …   0.101817    0.100802    0.143131
 -0.000374575   1.0              0.463328    0.473686    0.30033
  0.315817      0.112863         0.234097    0.230312    0.312048
 -0.121171     -0.0274317       -0.0231393  -0.0198914  -0.0180876
  0.0562746    -0.0740926       -0.0732009  -0.0709649  -0.0446842
 -0.110251     -0.0582016    …  -0.0873943  -0.0826741  -0.0647018
  0.0914291     0.448818         0.969039    0.961999    0.683481
  0.101817      0.463328         1.0         0.985738    0.704234
  0.100802      0.473686         0.985738    1.0         0.706248
  0.143131      0.30033          0.704234    0.706248    1.0

## One simulation

In [22]:
# simulate data
m = 1
p = 500
k = 10 # number of causal groups
n = 250 # sample size
μ = zeros(p)
# Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

# define true groups and true covariance
groupsizes = [5 for i in 1:div(p, 5)] 
groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
Σ = simulate_block_covariance(groups, 0.75, 0.0)

# define groups empirically
nrep = 1
groups, group_reps = id_partition_groups(X, nrep=nrep)
unique_groups = unique(groups)

# simulate X
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# simulate βtrue: SNPs in causal groups will share the same effect 
δ = 2
βtrue = zeros(p)
shuffle!(unique_groups)
causal_groups = unique_groups[1:k]
for g in causal_groups
    β_idx = findall(x -> x == g, groups)
    effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
    βtrue[β_idx] .= effect_size
end

# simulate y
ϵ = randn(n)
y = X * βtrue + ϵ
correct_snps = findall(!iszero, βtrue)

# fully general me
me_t = @elapsed me = modelX_gaussian_group_knockoffs(
    X, :maxent, groups, μ, Σ, 
    m = m,
    niter = 10,
    tol = 0.01,    # convergence tolerance
    verbose=false, # whether to print informative intermediate results
)
me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_ssum = sum(abs.(me_ko_filter.ko.S))
@show me_power, me_fdr

# representative ME knockoffs
rme = modelX_gaussian_rep_group_knockoffs(
    X, :maxent, μ, Σ, groups, group_reps,
    m = m, 
    nrep = nrep
);
rme_ko_filter = fit_lasso(y, rme)
discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
rme_power = round(TP(causal_groups, discovered_groups), digits=3)
rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
@show rme_power, rme_fdr

(me_power, me_fdr) = (0.9, 0.1)
(rme_power, rme_fdr) = (1.0, 0.167)


(1.0, 0.167)

### Hierarchica clustering, target FDR = 0.1, nrep=2, m = 1

In [24]:
fdr_hat = 0.0
nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 1
    p = 500
    k = 10 # number of causal groups
    n = 1250 # sample size
    μ = zeros(p)
    # Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

    # define true groups and true covariance
    groupsizes = [5 for i in 1:div(p, 5)] 
    groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
    Σ = simulate_block_covariance(groups, 0.75, 0.0)

    # define groups empirically
    nrep = 2
    groups, group_reps = hc_partition_groups(Σ, nrep=nrep)
    unique_groups = unique(groups)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # simulate βtrue: SNPs in causal groups will share the same effect 
    δ = 1
    βtrue = zeros(p)
    shuffle!(unique_groups)
    causal_groups = unique_groups[1:k]
    for g in causal_groups
        β_idx = findall(x -> x == g, groups)
        effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
        βtrue[β_idx] .= effect_size
    end

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ
    correct_snps = findall(!iszero, βtrue)

    # fully general me
    me_t = @elapsed me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims

(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.231)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)


0.0686

### interpolative decomposition, target FDR = 0.1, nrep=2, m = 5

In [29]:
fdr_hat = 0.0
nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = 500
    k = 10 # number of causal groups
    n = 1250 # sample size
    μ = zeros(p)
    # Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

    # define true groups and true covariance
    groupsizes = [5 for i in 1:div(p, 5)] 
    groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
    Σ = simulate_block_covariance(groups, 0.75, 0.0)

    # define groups empirically
    nrep = 2
    groups, group_reps = id_partition_groups(X, nrep=nrep)
    unique_groups = unique(groups)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # simulate βtrue: SNPs in causal groups will share the same effect 
    δ = 1
    βtrue = zeros(p)
    shuffle!(unique_groups)
    causal_groups = unique_groups[1:k]
    for g in causal_groups
        β_idx = findall(x -> x == g, groups)
        effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
        βtrue[β_idx] .= effect_size
    end

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ
    correct_snps = findall(!iszero, βtrue)

    # fully general me
    me_t = @elapsed me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims

(me_power, me_fdr) = (1.0, 0.167)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.286)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.167)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.167)


0.0774

### interpolative decomposition, target FDR = 0.25, nrep=5, m = 5

In [31]:
fdr_hat = 0.0
nsims = 20 # 10 showed slightly inflated FDR, try 20
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = 500
    k = 10 # number of causal groups
    n = 1250 # sample size
    μ = zeros(p)
    # Σ = Matrix(SymmetricToeplitz(0.9.^(0:(p-1))))

    # define true groups and true covariance
    groupsizes = [5 for i in 1:div(p, 5)] 
    groups = vcat([i*ones(g) for (i, g) in enumerate(groupsizes)]...) |> Vector{Int}
    Σ = simulate_block_covariance(groups, 0.75, 0.0)

    # define groups empirically
    nrep = 5
    groups, group_reps = id_partition_groups(X, nrep=nrep)
    unique_groups = unique(groups)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # simulate βtrue: SNPs in causal groups will share the same effect 
    δ = 1
    βtrue = zeros(p)
    shuffle!(unique_groups)
    causal_groups = unique_groups[1:k]
    for g in causal_groups
        β_idx = findall(x -> x == g, groups)
        effect_size = rand(-1:2:1) * rand(Uniform(δ/2, δ)) / length(β_idx)
        βtrue[β_idx] .= effect_size
    end

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ
    correct_snps = findall(!iszero, βtrue)

    # fully general me
    me_t = @elapsed me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, X, me, debias=:lasso)
    me_power = round(TP(causal_groups, me_ko_filter.βs[4], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[4], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[4])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims

(me_power, me_fdr) = (1.0, 0.333)
(rme_power, rme_fdr) = (1.0, 0.524)
(me_power, me_fdr) = (1.0, 0.524)
(rme_power, rme_fdr) = (1.0, 0.412)
(me_power, me_fdr) = (1.0, 0.412)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.167)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.375)
(rme_power, rme_fdr) = (1.0, 0.333)
(me_power, me_fdr) = (1.0, 0.375)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.333)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.375)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.231)
(me_power, me_fdr) = (1.0, 0.091)
(rme_power, rme_fdr) = (1.0, 0.167)
(me_power, me_fdr) = (1.0, 0.231)
(rme_power, rme_fdr) = (1.0, 0.0)
(me_power, me_fdr) = (1.0, 0.0)
(rme_power, rme_fdr) = (1.0, 0.091)
(me_power, me_fdr) = (1.

0.22034999999999996